In [1]:
############### Imports ###############
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import statsmodels.api as sm 

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import make_pipeline
from sklearn_pandas import DataFrameMapper


In [2]:
############### Data ###############
df1 = pd.read_csv("/Users/biancaorozco/Desktop/Metis/week02/project-02/airfare_project2/expedia_10_8_2019.csv")
df2 = pd.read_csv("/Users/biancaorozco/Desktop/Metis/week02/project-02/airfare_project2/expedia_10_9_2019.csv")
df3 = pd.read_csv("/Users/biancaorozco/Desktop/Metis/week02/project-02/airfare_project2/expedia_10_10_2019.csv")

frames = [df1, df2, df3]
na_df = pd.concat(frames)
print(na_df.shape)
na_df.sample(15, random_state=40)

(3183, 10)


/Users/biancaorozco/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,airline,arrival_airport,arrival_time,date,departure_airport,departure_time,duration,id_flight,number_stops,prices
168,American Airlines,San Francisco SFO,7:11pm,10/9/2019,Atlanta ATL,3:20pm,4h 51m,13.0,(1 stop),$376
1237,United,San Francisco SFO,9:31am,10/8/19,Seattle SEA,7:18am,2h 13m,11.0,(Nonstop),$336
711,United,Los Angeles LAX,7:40pm,10/9/2019,Dallas DFW,2:15pm,6h 25m,25.0,(1 stop),$379
2,JetBlue Airways,New York JFK,1:10pm,10/8/19,Atlanta ATL,10:56am,2h 14m,2.0,(Nonstop),$294
1172,United,Denver DEN,12:06pm,10/8/19,Chicago ORD,10:15am,2h 51m,16.0,(Nonstop),$414
246,United,Chicago ORD,7:43pm,10/10/2019,Los Angeles LAX,10:45am,6h 58m,1.0,(Nonstop),$137
578,American Airlines,Chicago ORD,2:16pm,10/10/2019,Atlanta ATL,9:08am,6h 8m,25.0,(1 stop),$292
158,American Airlines,Dallas DFW,12:31am,10/8/19,Chicago ORD,10:09pm,2h 22m,3.0,(Nonstop),$399
671,Delta,Chicago ORD,5:52pm,10/8/19,New York JFK,8:05am,2h 53m,32.0,(Nonstop),$269
371,United,Dallas DFW,1:18pm,10/8/19,Denver DEN,5:30am,6h 48m,41.0,(1 stop),$379


In [3]:
############### Functions ###############
## Duration Time to Minutes
def duration_to_min(duration):
    time = duration.split()
    try:
        minutes = int(time[0])*60 + int(time[1])
        return minutes
    except:
        return None

## 12-hr Time to Minutes
def time_to_min(times):
    ztime = times.zfill(7)
    spaced_time = ztime[:5] + ' ' + ztime[5:]
    split = spaced_time.split()
    try:
        if split[2] == 'pm':
            minutes = (int(split[0])+12)*60 + int(split[1])
        else:
            minutes = int(split[0])*60 + int(split[1])
        return minutes
    except:
        return spaced_time

In [4]:
############### Cleaning Data ###############
## Dropping rows with NaNs
na_df.isna().sum()
df = na_df.dropna()
df.isna().sum()

airline              0
arrival_airport      0
arrival_time         0
date                 0
departure_airport    0
departure_time       0
duration             0
id_flight            0
number_stops         0
prices               0
dtype: int64

In [9]:
df['id_flight'] = pd.to_numeric(df.id_flight, downcast='signed')

## Replacing Number of Stops
df['number_stops'].replace('(Nonstop)', value=int('0'), regex=True, inplace=True)
df['number_stops'].replace('(1 stop)', value=int('1'), regex=True, inplace=True)
df['number_stops'].replace('(2 stop)', value=int('2'), regex=True, inplace=True)
df['number_stops'].replace('(3 stop)', value=int('3'), regex=True, inplace=True)
            
## Removing $ and , from Prices and Convert to Numeric
df['prices'].replace('\$', value='', regex=True, inplace=True)
df['prices'].replace(',', value='', regex=True, inplace=True)
df['prices'] = pd.to_numeric(df.prices)

## Removing h & m from Duration Times
df['duration'].replace('h', value='', regex=True, inplace=True)
df['duration'].replace('m', value='', regex=True, inplace=True)

## Replacing : with _ in Departure and Arrival Times
df['departure_time'].replace(':', value=' ', regex=True, inplace=True)
df['arrival_time'].replace(':', value=' ', regex=True, inplace=True)

## Executing Cleaning Functions
df['departure_time'] = df['departure_time'].apply(time_to_min)
df['arrival_time'] = df['arrival_time'].apply(time_to_min)
df['duration'] = df['duration'].apply(duration_to_min)

/Users/biancaorozco/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/biancaorozco/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/biancaorozco/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [69]:
df.head()

,airline,arrival_airport,arrival_time,date,departure_airport,departure_time,duration,id_flight,number_stops,prices
0,Delta,New York JFK,780,10/8/19,Atlanta ATL,650,130,0,0,294
1,Delta,New York JFK,638,10/8/19,Atlanta ATL,506,132,1,0,294
2,JetBlue Airways,New York JFK,790,10/8/19,Atlanta ATL,656,134,2,0,294
3,Delta,New York JFK,1419,10/8/19,Atlanta ATL,1285,134,3,0,294
4,Delta,New York JFK,879,10/8/19,Atlanta ATL,1460,139,4,0,294


In [70]:
flights = df.select_dtypes(exclude=['object']).copy()
flights['airline']=df['airline']
flights.head()

,arrival_time,departure_time,duration,id_flight,number_stops,prices,airline
0,780,650,130,0,0,294,Delta
1,638,506,132,1,0,294,Delta
2,790,656,134,2,0,294,JetBlue Airways
3,1419,1285,134,3,0,294,Delta
4,879,1460,139,4,0,294,Delta


In [87]:
pd.get_dummies(df).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3064 entries, 0 to 897
Data columns (total 34 columns):
arrival_time                           3064 non-null int64
departure_time                         3064 non-null int64
duration                               3064 non-null int64
id_flight                              3064 non-null int8
number_stops                           3064 non-null int64
prices                                 3064 non-null int64
airline_Alaska Airlines                3064 non-null uint8
airline_American Airlines              3064 non-null uint8
airline_Boutique Air                   3064 non-null uint8
airline_Delta                          3064 non-null uint8
airline_Frontier Airlines              3064 non-null uint8
airline_JetBlue Airways                3064 non-null uint8
airline_Multiple Airlines              3064 non-null uint8
airline_Spirit Airlines                3064 non-null uint8
airline_Sun Country Airlines           3064 non-null uint8
airline_Un

In [79]:
flights2 = df.select_dtypes(exclude=['object']).copy()
flights2['departure_airport']=df['departure_airport']
flights2.head()

,arrival_time,departure_time,duration,id_flight,number_stops,prices,departure_airport
0,780,650,130,0,0,294,Atlanta ATL
1,638,506,132,1,0,294,Atlanta ATL
2,790,656,134,2,0,294,Atlanta ATL
3,1419,1285,134,3,0,294,Atlanta ATL
4,879,1460,139,4,0,294,Atlanta ATL


In [90]:
print(df['airline'].nunique(), 'airlines') # 10 airlines
print(df['departure_airport'].nunique(), 'departure airports')
print(df['arrival_airport'].nunique(), 'arrival airports')

10 airlines
8 departure airports
7 arrival airports


In [91]:
df

,airline,arrival_airport,arrival_time,date,departure_airport,departure_time,duration,id_flight,number_stops,prices
0,Delta,New York JFK,780,10/8/19,Atlanta ATL,650,130,0,0,294
1,Delta,New York JFK,638,10/8/19,Atlanta ATL,506,132,1,0,294
2,JetBlue Airways,New York JFK,790,10/8/19,Atlanta ATL,656,134,2,0,294
3,Delta,New York JFK,1419,10/8/19,Atlanta ATL,1285,134,3,0,294
4,Delta,New York JFK,879,10/8/19,Atlanta ATL,1460,139,4,0,294
...,...,...,...,...,...,...,...,...,...,...
893,United,Denver DEN,1077,10/10/2019,Chicago ORD,621,516,77,1,637
894,American Airlines,New York JFK,1080,10/10/2019,Dallas DFW,806,214,0,0,509
895,American Airlines,New York JFK,831,10/10/2019,Dallas DFW,549,222,1,0,509
896,United,San Francisco SFO,489,10/10/2019,Seattle SEA,360,214,0,0,140


In [92]:
############### Features ###############
X = df.loc[:,['id_flight', 'departure_time', 'arrival_time', 'airline', 'duration', 'number_stops', 'departure_airport', 'arrival_airport', 'date']]
y = df['prices']
X

,id_flight,departure_time,arrival_time,airline,duration,number_stops,departure_airport,arrival_airport,date
0,0,650,780,Delta,130,0,Atlanta ATL,New York JFK,10/8/19
1,1,506,638,Delta,132,0,Atlanta ATL,New York JFK,10/8/19
2,2,656,790,JetBlue Airways,134,0,Atlanta ATL,New York JFK,10/8/19
3,3,1285,1419,Delta,134,0,Atlanta ATL,New York JFK,10/8/19
4,4,1460,879,Delta,139,0,Atlanta ATL,New York JFK,10/8/19
...,...,...,...,...,...,...,...,...,...
893,77,621,1077,United,516,1,Chicago ORD,Denver DEN,10/10/2019
894,0,806,1080,American Airlines,214,0,Dallas DFW,New York JFK,10/10/2019
895,1,549,831,American Airlines,222,0,Dallas DFW,New York JFK,10/10/2019
896,0,360,489,United,214,0,Seattle SEA,San Francisco SFO,10/10/2019


In [101]:
## Dummy Variables for Airlines, Departure Airport, Arrival Airport, and Date
print(X['airline'].nunique(), 'airlines') # 10 airlines
print(X['departure_airport'].nunique(), 'departure airports')
print(X['arrival_airport'].nunique(), 'arrival airports\n')
X2 = pd.get_dummies(X)
X2.head()

10 airlines
8 departure airports
7 arrival airports



,id_flight,departure_time,arrival_time,duration,number_stops,airline_Alaska Airlines,airline_American Airlines,airline_Boutique Air,airline_Delta,airline_Frontier Airlines,...,arrival_airport_Chicago ORD,arrival_airport_Dallas DFW,arrival_airport_Denver DEN,arrival_airport_Los Angeles LAX,arrival_airport_New York JFK,arrival_airport_San Francisco SFO,arrival_airport_Seattle SEA,date_10/10/2019,date_10/8/19,date_10/9/2019
0,0,650,780,130,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,1,506,638,132,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,2,656,790,134,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,3,1285,1419,134,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,4,1460,879,139,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0


In [102]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=.2, random_state=12)

In [103]:
lm = LinearRegression()
kf = KFold(n_splits=5, # folds
            shuffle=True,  
            random_state=33)# arbitrary, but important
kf_scores = cross_val_score(lm, X_train, y_train, # estimator, features, target
                cv = kf, # 5 folds
                scoring='r2') # scoring metric
print(np.mean(kf_scores))

0.5922213976488859
